# Assignment 7 

## Grade: /100 pts

This notebook contains the questions for Assignment 7. 

Make sure to complete this assignment individually and appropriately reference all external code and documentation used. ***In order for your submission to be valid, you must adhere to the function definitions which have been made (failure to do so will result in a grade of 0). You must upload this completed Jupyter Notebook file as your submission (other file types are not permitted and will result in a grade of 0).*** You are responsible for selecting and importing additional packages.

### Preliminaries

Feel free to add any libraries to the Preliminaries. However, be mindful of every question's restrictions as some may exclude use of some functions.

In [106]:
## perform the necessary imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import recall_score, make_scorer, mean_squared_error, confusion_matrix, precision_score, roc_curve, auc, accuracy_score, f1_score, roc_auc_score
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
import time

# Plotting
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Data set

In this Assignment you need to download and use "Dataset.csv". 

This dataset is a modified dataset from Kaggle datasets called "Lower Back Pain Symptoms Dataset". Lower back pain can be caused by a variety of problems with any parts of the complex, interconnected network of spinal muscles, nerves, bones, discs or tendons in the lumbar spine.
This data set is about to identify/label a person as abnormal or normal using collected physical spine details/data.



### Question 1: Load Datasets (15pts)

A) Load the "Dataset.csv" file.

B) Encode the output classes `Label` (0: Normal, 1: Abnormal) and separate inputs and outputs (features and target). (2 pts)

C) Split the data into equals-sized training and test sets. Use a random_state = 42, and ensure the `balanced distribution` of labels when splitting data.  

D) How many observations do you have in your training set?  

E) How many observations for each class in your training set?

F) Z-standarize the input features of the training and test sets.

### Question 2: Logistic Regression (15 pts)

A) Build a L1-regularized logistic regression model to all the training data, and then get the predicted labels for each item of the test set. Tip: use the 'saga' solver for L1 regularization.

B) Print out the precision, recall, and F1-score of the test set.

C) Print out the model execution time (both training and test time) in milliseconds. Please keep two decimal places.

D) Plot ROC curve and report the area under the ROC curve for the test data set. 


### Question3: Neural Network 


### Q3a) Building model (15 pts)

Build a simple neural network model (NN_model1) using PyTorch packages with the features in the data set as the input units and two output units for the two output classes:

* Use a LogSigmoid as your output non-linearity.
* Use the Cross-entropy loss as a training criterion. 
* Use Stochastic gradient descent optimizer with a learning rate of 0.01. 
* Run the optimization for 8000 iterations and record the loss for each iteration. 
* Plot the loss versus iterations.

### Q3b) Prediction (20 pts)

Now use your trained model (NN_model1) to make predictions on the test set.

A) Print out the precision, recall, and F1-score of the test set.

B) Print out the model execution time (both training and test time) in milliseconds. Please keep two decimal places.

C) Plot ROC curve and report the area under the ROC curve for the test data set. 

### Q3c) Adding hidden layers (15 pts)
Change the neural network (NN_model2) and add two hidden layers with 100 and 60 units, respectively. Use the LogSigmoid non-linearity for the hidden layers. Leave all the other parameters the same as for Question 3a. Again train for 8000 iterations and plot the loss as a function of the iteration. 

### Q3d) Prediction and model selection (20 pts)
Now use your trained model in Question 3c (NN_model2) to make predictions on the test set.

A) Print out the precision, recall, and F1-score of the test set.

B) Print out the model execution time (both training and test time) in milliseconds. Please keep two decimal places.

C) Plot ROC curve and report the area under the ROC curve for the test data set. 

__Written answer:__ Compare this model (NN_model2) to the results from Question 2 (Logistic Regression) and 3b (NN_model1), what do you conclude? 

__Written answer:__ 